This code will create a STAC catalog and add several items from a local folder

How do I put the date of image acquisition for an imagery product?  I don't think there is an automated way to do it. I need to manually input it. But how? Maybe I need to add metadata to imagery products within Cyverse and then read that metadata with pystack? How do I write metadata in Cyverse? 

In [1]:
%pip install rasterio shapely pystac

Note: you may need to restart the kernel to use updated packages.


In [2]:
#Additional Metadata to add to items
platform ="DJI Phantom 4 RTK"
license = "CC-BY-SA-4.0"

#input the date and time of original imagery acquisition
#dating assets will be a manual process. 
from datetime import datetime
year = 2019
month = 5
day = 25
hour = 12
    
#Create datetime object
user_datetime = datetime(year, month, day, hour)

data_type = 'uint8'
mission_description ="The imagery was part of the Ecostate Mapping project of 2019 at Santa Rita Experimental Range"


In [3]:
from pathlib import Path
import pystac
import os

catalog = pystac.Catalog(
    id='Cyverse Remotely Sensed Imagery STAC Catalog',
    description='This catalog includes all of the imagery assets the exist in Cyverse Data Store',
    stac_extensions=['https://stac-extensions.github.io/projection/v1.0.0/schema.json',
                    'https://stac-extensions.github.io/processing/v1.1.0/schema.json']
)

This following code is a Python function called "get_bbox_and_footprint" that takes in a dataset (either from rasterio or rioxarray) and returns a tuple containing the bounding box of the dataset and the footprint of the dataset. The bounding box is represented as a list of 4 coordinates [left, bottom, right, top] and the footprint is represented as a shapely Polygon object.

First, the code imports the Polygon and mapping functions from the shapely.geometry library, and the rasterio library.

The function starts by extracting the bounds of the input dataset using the bounds attribute. The code then checks if the bounds are of type rasterio.coords.BoundingBox or not. If they are, it assigns the left, bottom, right and top coordinates of the bounding box to the bbox variable. If not, it creates a list of floating point numbers by iterating over the bounds and assigns it to the bbox variable.

Finally, the code creates a Polygon object using a list of the four corner coordinates of the bounding box. The footprint is obtained by mapping the Polygon object.

The function returns a tuple containing the bounding box and the footprint of the dataset.

In [4]:
import rasterio

#a function to truncate spatial resolution to 3 decimal places
def trun_n_d(num,n):
    num_s = str(num)
    if 'e' in num_s or 'E' in num_s:
        return '{0:.{1}f}'.format(num,n)
    i,p,d = num_s.partition('.')
    return '.'.join([i,(d+'0'*n)[:n]])

#function to get the spatial resolution of a raster
def spatial_resolution(raster):
    """extracts the XY Pixel Size"""
    t = raster.transform
    x = t[0]
    y = -t[4]
    x_trunc = trun_n_d(x, 3)
    y_trunc = trun_n_d(y, 3)
    return x_trunc, y_trunc

In [5]:
from shapely.geometry import Polygon, mapping
import rasterio
import rasterio.warp
import rasterio.features

#This creates a function called 'get_bbox_and_footprint' 
def get_bbox_and_footprint(dataset):

    # create the bounding box it will depend if it comes from rasterio or rioxarray
    bounds = dataset.bounds
    
    bounds = rasterio.warp.transform_bounds(dataset.crs, 'EPSG:4326', 
                                            bounds.left, bounds.bottom, bounds.right, bounds.top)
    bounds = rasterio.coords.BoundingBox(bounds[0], bounds[1], bounds[2], bounds[3])
    
    #from rasterio.warp import transform_bounds, transform_geom
    #bounds = rasterio.warp.transform_geom(
         #   dataset.crs, 'EPSG:4326', bounds, precision=6)
    
    if isinstance(bounds, rasterio.coords.BoundingBox):
        bbox = [bounds.left, bounds.bottom, bounds.right, bounds.top]
    else:
        bbox = [float(f) for f in bounds()]

    # create the 4 corners of the footprint
    footprint = Polygon([
        [bbox[0], bbox[1]],#left bottom
        [bbox[0], bbox[3]],#left top
        [bbox[2], bbox[3]],#right top
        [bbox[2], bbox[1]] #right bottom
    ])

    return bbox, mapping(footprint)

In [7]:
from pystac.extensions.projection import ProjectionExtension
from pystac.extensions.raster import RasterExtension

#get a list of geospatial files (in this case located in Cyverse data store)
folder = Path('/data-store/iplant/home/jgillan/STAC_drone')
files = list(folder.rglob('*.tif'))#find files that end with .tif



#loop through each .tif item in the folder and do several things
all_items = []
for file in files:

    # open the individual file with rasterio
    ds = rasterio.open(file)
    #print(file)
    #apply the function to get the bounding box (left, bottom, right, top) and make a footprint rectangle
    bbox, footprint = get_bbox_and_footprint(ds)
    
    #Extract the spatial resolution (gsd) of the image product using the function 'spatial_resolution'.
    x_res,y_res = spatial_resolution(ds)
    
    #the ID (name) for each indivual file
    idx = file.stem
    
    
    #Create a STAC item for each individual file 
    item = pystac.Item(id=idx,
                 geometry=footprint,
                 bbox=bbox,
                 datetime=user_datetime,
                 stac_extensions=['https://stac-extensions.github.io/projection/v1.0.0/schema.json',
                                 'https://stac-extensions.github.io/processing/v1.1.0/schema.json'],
                 properties={"gsd": x_res,
                            "platform":platform,
                            "license":license,
                            "mission":mission_description})
    
    # Adding the projection extension to each item 
    asset_ext = ProjectionExtension.add_to(item)
    ProjectionExtension.ext(item).apply(ds.crs.to_epsg(),
                                        shape = ds.shape,
                                        bbox = bbox,
                                        geometry = footprint)
                                        #transform = [float(getattr(ds.transform, letter)) for letter in 'abcdef']
                                        #)
  

    #raster_ext = RasterExtension.add_to(item)
   # ItemRasterExtension.ext(item).apply(datatype = data_type)
        
    #add the asset to each STAC item                      
    item.add_asset(
        key='geotiff',
        asset=pystac.Asset(
            href=file.as_posix(),
            media_type=pystac.MediaType.GEOTIFF,
            extra_fields=asset_ext
        )
    )
    
    # Add each STAC item to the list
    all_items.append(item)
    

ImportError: cannot import name 'ItemRasterExtension' from 'pystac.extensions.raster' (/opt/conda/lib/python3.10/site-packages/pystac/extensions/raster.py)

In [ ]:
item_extents = pystac.Extent.from_items(all_items)


collection = pystac.Collection(id='Drone_Imagery_Collection',
                               description='Drone_Imagery',
                               extent=item_extents,
                               license='CC-BY-SA-4.0')

for item in all_items:
    #add each STAC item to the STAC collection
    collection.add_item(item)
    
catalog.add_child(collection)

In [ ]:
#print the number of STAC items that were added to the STAC catalog    
print(len(list(collection.get_items())))

#Describe the items in the STAC catalog
catalog.describe()

In [ ]:
#output directory for STAC catalog
stac_directory="/data-store/iplant/home/jgillan/STAC_drone"

catalog.normalize_hrefs(os.path.join(stac_directory, 'stac_test105'))#name of the new directory

catalog.make_all_asset_hrefs_relative()
catalog.save(catalog_type=pystac.CatalogType.SELF_CONTAINED)